In [2]:
import pandas as pd
import sqlite3
pd.set_option('display.max_columns', None)

In [1]:
def clean_data(csv_loc):
    df = pd.read_csv(csv_loc)
    df = df.drop('IncidentTopSRS_UCR', axis=1)
    df.rename(columns={'CrimeAgainst': 'NIBRSCat',
                       'NIBRS': 'NIBRSCode',
                       'NIBRSCategory':'NIBRSOffenseType',
                       'SRS_UCR':'UCR_SRS',
                       'OccurredFromTime':'TimeOccurred',
                       'Offense':'SLMPDOffense',
                       'FelMisdCit':'CrimeGrade',
                       'IncidentLocation':'PrimaryLocation',
                       'IntersectionOtherLoc':'SecondaryLocation',
                       'NbhdNum':'NeighborhoodNum',
                       'IncidentSupplemented':'Supplemented',
                       'LastSuppDate':'SupplementDate'}, inplace=True)
    
    ordered_cols = ['IncidentNum', 'IncidentDate', 'TimeOccurred', 'SLMPDOffense',
                    'NIBRSCode', 'NIBRSCat', 'NIBRSOffenseType', 'UCR_SRS', 'CrimeGrade',
                    'PrimaryLocation', 'SecondaryLocation', 'District', 'Neighborhood',
                    'NeighborhoodNum', 'Latitude', 'Longitude', 'Supplemented',
                    'SupplementDate', 'VictimNum', 'FirearmUsed', 'IncidentNature']
    df = df[ordered_cols]
    
    supp_df = df[df['Supplemented'] == 'Yes']
    unfound_df = df[(df['Supplemented'].isna()) & (df['SLMPDOffense'] == 'UNFOUNDED INCIDENT')]
    new_df = df[df['Supplemented'] == 'No']

    if len(df) != len(supp_df) + len(unfound_df) + len(new_df):
        print("Something doesn't add up")




    
    # supp_df transformations:
    
    # You can directly update the SQLite database table using to_sql() method
    supp_df.to_sql('df_temp', conn, if_exists='replace', index=False)
    
    # Update corresponding rows in the 'crime_data' table with values from jan_df
    # Construct the update query dynamically
    update_query = '''
        UPDATE crime_data
        SET {}
        WHERE EXISTS (SELECT 1 FROM df_temp WHERE crime_data.IncidentNum = df_temp.IncidentNum)
    '''.format(', '.join([f"{col} = (SELECT {col} FROM df_temp WHERE crime_data.IncidentNum = df_temp.IncidentNum)" for col in ordered_cols]))
    
    # Construct the insert query for rows not existing in crime_data
    insert_query = '''
        INSERT INTO crime_data ({})
        SELECT {}
        FROM df_temp
        WHERE NOT EXISTS (SELECT 1 FROM crime_data WHERE crime_data.IncidentNum = df_temp.IncidentNum)
    '''.format(', '.join(ordered_cols), ', '.join([f"df_temp.{col}" for col in ordered_cols]))
    
    # Execute the update query
    conn.execute(update_query)
    
    # Execute the insert query
    conn.execute(insert_query)
    
    # Drop temporary table
    conn.execute('DROP TABLE IF EXISTS df_temp')
    conn.commit()

    # Return updated table
    updated_df = pd.read_sql_query("SELECT * FROM crime_data", conn)

    

    return [df, supp_df, unfound_df, new_df, updated_df]

In [3]:
conn=sqlite3.connect('test_db.db')

In [4]:
jan_loc = 'uploads/Crime_01_2024.csv'
cleaned = clean_data(jan_loc)
clean_df, supp_df, unfound_df, new_df, updated_df = cleaned

In [5]:
updated_df

,Id,IncidentNum,IncidentDate,TimeOccurred,SLMPDOffense,NIBRSCode,NIBRSCat,NIBRSOffenseType,UCR_SRS,CrimeGrade,PrimaryLocation,SecondaryLocation,District,Neighborhood,NeighborhoodNum,Latitude,Longitude,Supplemented,SupplementDate,VictimNum,FirearmUsed,IncidentNature
0,0.0,21000002,2021-01-01,02:00.0,MISCELLANEOUS WEAPON VIOLATION,520,None,None,NaN,None,1700 CHESTNUT ST,None,4.0,36,None,38.629573,-90.204995,None,None,NaN,None,None
1,1.0,21000003,2021-01-01,09:00.0,MISCELLANEOUS WEAPON VIOLATION,520,None,None,NaN,None,4961 LACLEDE AVE,210,5.0,38,None,38.641221,-90.264226,None,None,NaN,None,None
2,2.0,21000003,2021-01-01,09:00.0,PROPERTY DAMAGE,290,None,None,NaN,None,4961 LACLEDE AVE,210,5.0,38,None,38.641221,-90.264226,None,None,NaN,None,None
3,3.0,21000007,2021-01-01,10:00.0,PROPERTY DAMAGE - 2ND DEGREE,290,None,None,NaN,None,6963 PERNOD AVE,None,2.0,9,None,38.598725,-90.310722,None,None,NaN,None,None
4,4.0,21000008,2021-01-01,20:00.0,MISCELLANEOUS WEAPON VIOLATION,520,None,None,NaN,None,3500 KINGSLAND CT,None,1.0,16,None,38.579494,-90.243210,None,None,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175451,NaN,24005524,2024-01-12,06:00:00,STEALING - ALL OTHER PROPERTY UNDER 570.030.5 (3),23D,Property,Theft From Building,6.0,F,3937 N 20TH ST,None,4.0,Hyde Park,65.0,38.663273,-90.204579,Yes,2024-02-15 08:31:49.0200000,NaN,No,Larceny - Criminal
175452,NaN,24005689,2024-01-27,11:05:00,STEALING - POSTAL OR DELIVERY ITEM,23H,Property,All Other Larceny,6.0,None,59 WILLMORE RD,None,2.0,St Louis Hills,8.0,38.577052,-90.307070,Yes,2024-02-21 08:35:04.7330000,NaN,No,Larceny - Criminal
175453,NaN,24006233,2024-01-25,10:14:00,STEALING - MOTOR VEHICLE/WATERCRAFT/AIRCRAFT &...,26A,Property,False Pretense/Swindle/Confidence Game,NaN,F,2210 S 7TH ST,None,3.0,Kosciusko,20.0,38.604615,-90.203040,Yes,2024-02-14 08:08:26.0000000,NaN,No,Fraud - Criminal
175454,NaN,24006728,2024-01-09,06:00:00,STEALING - $750 OR MORE,23H,Property,All Other Larceny,6.0,F,1700 WASHINGTON AVE,None,4.0,Downtown West,36.0,38.633288,-90.203490,Yes,2024-02-24 18:50:20.4800000,NaN,No,Larceny - Criminal


In [ ]:
df = pd.read_sql_query("SELECT * FROM crime_data", conn)

In [ ]:
df

In [ ]:
df.info()

In [6]:
conn.close()

In [ ]:
df = pd.read_csv('uploads/Crime2021-2023.csv')
df